### imports

In [ ]:
from IPython.display import display, Markdown
import openai
from top_secret import my_sk

from markdown import markdown
from weasyprint import HTML
from PyPDF2 import PdfReader

In [ ]:
### Convert pdf to markdown file 

In [ ]:

def pdf_to_markdown_simple(pdf_path, markdown_path):
    reader = PdfReader(pdf_path)
    full_text = ""

    # Extract text from each page
    for page in reader.pages:
        full_text += page.extract_text()

    # Save to a Markdown file
    with open(markdown_path, "w", encoding="utf-8") as md_file:
        md_file.write(full_text)

    print(f"Converted PDF to Markdown: {markdown_path}")


In [ ]:
pdf_to_markdown_simple("resumes/ankitk_updated2024.pdf", "resumes/ankitk_updated2024.md")

### 1) Import Resume & Job Description

In [ ]:
# Open and read the Markdown file
with open("resumes/ankitk_updated2024.md", "r", encoding="utf-8") as file:
    resume_string = file.read()

# display resume
display(Markdown(resume_string))

In [ ]:
# input job description
jd_string = input()

### 2) Construct Prompt Template

In [ ]:
prompt_template = lambda resume_string, jd_string : f"""
You are a professional resume optimization expert specializing in tailoring resumes to specific job descriptions. Your goal is to optimize my resume and provide actionable suggestions for improvement to align with the target role.

### Guidelines:
1. **Relevance**:  
   - Prioritize experiences, skills, and achievements **most relevant to the job description**.  
   - Remove or de-emphasize irrelevant details to ensure a **concise** and **targeted** resume.
   - Limit work experience section to 2-3 most relevant roles
   - Limit bullet points under each role to 2-3 most relevant impacts

2. **Action-Driven Results**:  
   - Use **strong action verbs** and **quantifiable results** (e.g., percentages, revenue, efficiency improvements) to highlight impact.  

3. **Keyword Optimization**:  
   - Integrate **keywords** and phrases from the job description naturally to optimize for ATS (Applicant Tracking Systems).  

4. **Additional Suggestions** *(If Gaps Exist)*:  
   - If the resume does not fully align with the job description, suggest:  
     1. **Additional technical or soft skills** that I could add to make my profile stronger.  
     2. **Certifications or courses** I could pursue to bridge the gap.  
     3. **Project ideas or experiences** that would better align with the role.  

5. **Formatting**:  
   - Output the tailored resume in **clean Markdown format**.  
   - Include an **"Additional Suggestions"** section at the end with actionable improvement recommendations.  

---

### Input:
- **My resume**:  
{resume_string}

- **The job description**:  
{jd_string}

---

### Output:  
1. **Tailored Resume**:  
   - A resume in **Markdown format** that emphasizes relevant experience, skills, and achievements.  
   - Incorporates job description **keywords** to optimize for ATS.  
   - Uses strong language and is no longer than **one page**.

2. **Additional Suggestions** *(if applicable)*:  
   - List **skills** that could strengthen alignment with the role.  
   - Recommend **certifications or courses** to pursue.  
   - Suggest **specific projects or experiences** to develop.
"""

In [ ]:
prompt = prompt_template(resume_string, jd_string)

### 3) Generate Resume with GPT-4o-mini

In [ ]:
# setup api client
##to be removed

#openai.api_key = {{}}

# make api call
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Expert resume writer"},
        {"role": "user", "content": prompt}
    ], 
    temperature = 0.7
)

# extract response
response_string = response.choices[0].message.content

### 4) Display Results

In [ ]:
# separate new resume from improvement suggestions
response_list = response_string.split("## Additional Suggestions")

In [ ]:
display(Markdown(response_list[0]))

#### save new resume

In [ ]:
# save as PDF
output_pdf_file = "resumes/resume_new.pdf"

# Convert Markdown to HTML
html_content = markdown(response_list[0])

# Convert HTML to PDF and save
HTML(string=html_content).write_pdf(output_pdf_file, stylesheets=['resumes/style.css'])

In [ ]:
# save as markdown
output_file = "resumes/resume_new.md"

with open(output_file, "w", encoding="utf-8") as file:
    file.write(response_list[0])

#### display suggestions for improvement

In [ ]:
display(Markdown(response_list[1]))